In [1]:
from pathlib import Path

import flopy
import pastas as ps
from pastas_plugins import modflow as ppmf
import pandas as pd

bindir = Path("bin")
if not (bindir / "mf6").exists():
    bindir.mkdir(parents=True, exist_ok=True)
    flopy.utils.get_modflow(bindir, repo="modflow6")

In [2]:
data = ps.load_dataset("collenteur_2021")

head = wagna = data["wagna"]["head [m]"].dropna().rename("head")
evap = data["wagna"]["evaporation [mm/d]"].multiply(1e-3).rename("evaporation").dropna()
prec = data["wagna"]["precipitation [mm/d]"].multiply(1e-3).rename("precipitation").loc[evap.index]

In [3]:
ml = ps.Model(head, name="mftest")
mfml = ppmf.ModflowModel(
    ml,
    exe_name="bin/mf6",
    sim_ws="mftest",
    tmin=prec.index[0],
    tmax=prec.index[-1],
)
rch = ppmf.ModflowRch(prec, evap)
ghb = ppmf.ModflowGhb()
mfml.add_modflow_package([rch, ghb])
ml.add_stressmodel(mfml)

In [4]:
mfml.simulate(p=ml.parameters.initial.values)

MFDataException: An error occurred in model "mfsm" package "rch". The error occurred while None in the "None" method.
Additional Information:
(1) Error occurred while adding dataset "stress_period_data" to block "period"